# BP-Weight

This program requires the following files:

    Gene-Disease Associations, All GO IDs.csv
    GO-BP ID Ancestor Functions.ipynb
    go.obo

The program outputs the following file:

    BP-Weight.csv
 
This program takes about 40 minutes and may seem unresponsive.

## Define default filenames

In [1]:
# Use os module to access files in other directories.
from os import path

# Default file names.
gene_disease_associations_path = path.abspath(
    '../Gene-Disease Associations/Gene-Disease Associations, All GO IDs.csv')

## Define a function to open the BP-Weight scores file

In [2]:
# Import the pandas module for opening files.
import pandas

def get_disease_bp_terms(filename):
    '''Open the gene-disease associations file.
    
    Parameters:
    filename (str): The filename to work with.
    dtype: The data type.
    header (int): The row to serve as header.
    usecols (str): The columns to use.
    '''
    disease_bp_terms = pandas.read_csv(
        filename,
        dtype = str,
        header = 0,
        usecols = ['DB ID', 'Disease', 'GO-BP ID'])
    
    # Return the opened file object.
    return disease_bp_terms

## Define a function to open gene-disease association files

In [3]:
# Import the pandas module for opening files.
import pandas

def get_disease_bp_terms(filename):
    '''Open the gene-disease associations file.
    
    Parameters:
    filename (str): The file to work with.
    header (int): The row to serve as header.
    usecols (str): The columns to use.
    '''
    disease_bp_terms = pandas.read_csv(
        filename,  
        header = 0,
        usecols=['DB ID', 'Disease', 'GO-BP ID'])  
    
    # Return the opened file object.
    return disease_bp_terms

## Open the gene-disease associations files to access the BP terms

In [4]:
# Open gene-disease file with all GO IDs.
disease_bp_terms = get_disease_bp_terms(
    gene_disease_associations_path)

### Display the contents of the gene-disease associations file

In [5]:
# For visualization only: may delete code line.
disease_bp_terms

,DB ID,Disease,GO-BP ID
0,114500,Colorectal cancer with chromosomal instability...,GO:0009895 | GO:0006417 | GO:0050918 | GO:0001...
1,114480,"Breast cancer, somatic | {Breast cancer, prote...",GO:0009895 | GO:0006417 | GO:0070201 | GO:0030...
2,125853,"Diabetes mellitus, noninsulin-dependent, late ...",GO:0006417 | GO:0070201 | GO:0071704 | GO:0072...
3,611162,"{Malaria, resistance to} | {Malaria, protectio...",GO:0009895 | GO:0006417 | GO:0070201 | GO:0030...
4,167000,"Ovarian cancer, somatic",GO:0009895 | GO:0006417 | GO:0070201 | GO:0071...
...,...,...,...
5410,235550,Hepatic venoocclusive disease with immunodefic...,GO:0008150 | GO:0006357 | GO:0044403 | GO:0016...
5411,615544,?Periventricular nodular heterotopia 6,GO:0008150 | GO:0007275 | GO:0032501 | GO:0048...
5412,234050,"Trichothiodystrophy 4, nonphotosensitive",GO:0008150 | GO:0009987 | GO:0051301 | GO:0007049
5413,614700,"Immunodeficiency, common variable, 8, with aut...",GO:0033036 | GO:0008150 | GO:0051179 | GO:0008104


## Define a function that creates a dictionary and stores the number of diseases annotated with a GO ID

For a specific GO ID, increase the disease count every time the same GO ID is found.

In [6]:
def count_bp_associations(disease_bp_terms):
    '''Return a dictionary with the number of diseases 
    annotated with a GO ID.
    
    Parameters:
    disease_bp_terms: Pandas data frame containing the gene-disease
    associations file. The file must have a 'GO-BP ID' column.
    '''
    # Create a dictionary to count diseases associated to a GO ID.
    bp_associations = {}

    # Iterate thru every row in the gene-disease associations file.
    for index, row in disease_bp_terms.iterrows():

        # Get the GO-BP IDs and convert them into a list.
        go_ids = row['GO-BP ID'].split(' | ')

        # Iterate through every GO ID in the list.
        for go_id in go_ids:
            
            try:

                # Increase number of diseases annotated with GO ID.
                bp_associations[go_id] += 1

            except KeyError:

                # GO ID key does not exist, so create key.
                bp_associations[go_id] = 1
        
    # Return dictionary with number of diseases associated to GO IDs.
    return bp_associations

## Creates a dictionary that stores the number of diseases annotated with a GO ID

In [7]:
# Get number of annotated diseases from file with all GO IDs.
bp_associations = count_bp_associations(disease_bp_terms)

### Display the number of diseases annotated with a GO ID

In [8]:
# For visualization only: may delete code line.
pandas.DataFrame.from_dict(
    bp_associations, orient = 'index', columns = ['Diseases'])

,Diseases
GO:0009895,193
GO:0006417,112
GO:0050918,29
GO:0001649,88
GO:1901873,7
...,...
GO:0140014,1
GO:0050911,1
GO:0045137,1
GO:1901962,1


## Define a function to find the ***BP-weight*** of each biological process

Where the ***BP-weight*** is found using the formula 

$$ W(p) = 1 - \sqrt{\dfrac{count_p}{n}}$$

where the weight $W$ is a function of a biological process $p$, $count_p$ is the number of diseases annotated with the biological process $p$, and $n$ is the total number of diseases in the data set.

In [9]:
# Import math module to take square root of values.
import math

def get_bp_weight(bp_associations, total_disease_number):
    '''Return a dictionary with weighted biological processes 
    (BP terms) based on the BP-weight formula. 
    
    Parameters:
    bp_associations (dict): Dictionary with GO IDs (the keys) and the
    number of diseases annotated with each GO ID (the values).
    total_disease_number (int): Value representing the total sum
    of diseases in the data set. For example, there could be about 
    6000 diseases in the data set, but about 4000 after merging 
    diseases with the same gene symbols, database IDs, etc. 
    '''
    # Create a dictionary to store the weight of each BP term.
    weight_dict = {}
    
    # Iterate thru every BP term (the keys) in the dictionary.
    for bp in bp_associations:
        
        # Get the number of diseases annotated with BP term.
        count = bp_associations[bp]
        
        # Apply weight formula and store the weight of the BP term.
        weight_dict[bp] = 1 - math.sqrt(count / total_disease_number)
        
    # Return dictionary with BP-weights.
    return weight_dict

## Find the weight of each biological process

In [10]:
# Get the total number of diseases in the data set.
diseases_number = len(disease_bp_terms)

# Get the weight of each BP for file with all GO IDs.
weight = get_bp_weight(bp_associations, diseases_number)

### Display the weight of each biological process

In [11]:
# For visualization only: may delete code line.
pandas.DataFrame.from_dict(
    weight, orient = 'index', columns = ['Weight'])

,Weight
GO:0009895,0.811210
GO:0006417,0.856183
GO:0050918,0.926819
GO:0001649,0.872520
GO:1901873,0.964046
...,...
GO:0140014,0.986411
GO:0050911,0.986411
GO:0045137,0.986411
GO:1901962,0.986411


## Create BP-weight score file without GO ID ancestors

In [12]:
# Get 'DB ID' and 'Disease' columns for BP-weight score file.
weight_score = disease_bp_terms[['DB ID', 'Disease']]

### Display BP-weight score file

In [13]:
# For visualization only: may delete code line.
weight_score

,DB ID,Disease
0,114500,Colorectal cancer with chromosomal instability...
1,114480,"Breast cancer, somatic | {Breast cancer, prote..."
2,125853,"Diabetes mellitus, noninsulin-dependent, late ..."
3,611162,"{Malaria, resistance to} | {Malaria, protectio..."
4,167000,"Ovarian cancer, somatic"
...,...,...
5410,235550,Hepatic venoocclusive disease with immunodefic...
5411,615544,?Periventricular nodular heterotopia 6
5412,234050,"Trichothiodystrophy 4, nonphotosensitive"
5413,614700,"Immunodeficiency, common variable, 8, with aut..."


## Create a square matrix to store BP-weight scores 

In [14]:
# Form square matrix by using the transpose (no GO ID ancestors).
weight_score = pandas.concat([weight_score, 
                              weight_score.transpose()])

### Display BP-weight score file

In [15]:
# For visualization only: may delete code line.
weight_score

,DB ID,Disease,0,1,2,3,4,5,6,7,...,5405,5406,5407,5408,5409,5410,5411,5412,5413,5414
0,114500.0,Colorectal cancer with chromosomal instability...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,114480.0,"Breast cancer, somatic | {Breast cancer, prote...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,125853.0,"Diabetes mellitus, noninsulin-dependent, late ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,611162.0,"{Malaria, resistance to} | {Malaria, protectio...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,167000.0,"Ovarian cancer, somatic",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5412,234050.0,"Trichothiodystrophy 4, nonphotosensitive",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5413,614700.0,"Immunodeficiency, common variable, 8, with aut...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5414,618425.0,Neurodevelopmental disorder with impaired spee...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DB ID,NaN,NaN,114500,114480,125853,611162,167000,114550,211980,601626,...,609432,140000,228600,617175,176305,235550,615544,234050,614700,618425


## Get the BP terms associated to each disease based on the disease's index

In [16]:
# Get dictionary from the gene-disease file with all GO IDs.
# Split each string and convert the resulting list into a set.
# Then store every set into a dictionary.
bp_term_dict = disease_bp_terms['GO-BP ID'].apply(
    lambda term: set(term.split(' | '))).to_dict()

### Display dictionary with row indexes as keys and BP term lists as values

In [17]:
# For visualization only: may delete code line.
pandas.DataFrame.from_dict(bp_term_dict, orient = 'index')

,0,1,2,3,4,5,6,7,8,9,...,2374,2375,2376,2377,2378,2379,2380,2381,2382,2383
0,GO:0010641,GO:0010917,GO:0001894,GO:0022602,GO:0036148,GO:0060562,GO:0110020,GO:0051347,GO:0009605,GO:0060967,...,GO:0014068,GO:1905898,GO:0050863,GO:0051123,GO:0046889,GO:0048762,GO:0032940,GO:0006886,GO:0030282,GO:1990403
1,GO:0001894,GO:0022602,GO:0048168,GO:0051347,GO:0009605,GO:0060967,GO:0010595,GO:0051092,GO:1990776,GO:0008219,...,None,None,None,None,None,None,None,None,None,None
2,GO:0001894,GO:0060562,GO:0051347,GO:0009605,GO:0060967,GO:0008219,GO:0044267,GO:0006140,GO:0044265,GO:0034103,...,None,None,None,None,None,None,None,None,None,None
3,GO:0051347,GO:0009605,GO:0060967,GO:0045843,GO:1900181,GO:0051092,GO:0008219,GO:0044267,GO:0006140,GO:0019222,...,None,None,None,None,None,None,None,None,None,None
4,GO:0001894,GO:0071869,GO:0060562,GO:0051347,GO:0009605,GO:0010595,GO:0008219,GO:0044267,GO:0060479,GO:0044265,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5410,GO:0008150,GO:0006357,GO:0044403,GO:0016032,GO:0044419,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5411,GO:0008150,GO:0032501,GO:0048856,GO:0032502,GO:0007275,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5412,GO:0008150,GO:0051301,GO:0007049,GO:0009987,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5413,GO:0033036,GO:0008150,GO:0008104,GO:0051179,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## Define a function that takes two row indexes (the diseases) and calculates the ***BP-weight score*** between the two diseases

Where the ***BP-weight score*** is defined as 

$$S(x,y) = \sum_{p_s \in remove\_ancestors(X \cap Y)} W(p_s)$$ 

where $X$ and $Y$ are the sets of biological processes associated with the corresponding diseases $x$ and $y$, the function $remove\_ancestors$ removes biological process parents, $p_s$ is a biological process shared by $x$ and $y$ that is not the parent of any shared biological processes, and $\sum W(p_s)$ is the summation of the weight of every shared biological process $p_s$. Note that the $remove\_ancestors$ function uses the [GOATOOLS](https://doi.org/10.1093/nar/gkw838) module for Python and [Gene Ontology](https://www.ebi.ac.uk/GOA/downloads) to determine parent and child biological processes.

### Import the function that will remove GO ID ancestors

In [18]:
# Run Jupyter Notebook that has the remove_ancestors function.
%run "GO-BP ID Ancestor Functions.ipynb"

D:\Documents\Research\Paper\Camera Ready\Programs\Gene-Disease Associations\go.obo: fmt(1.2) rel(2020-06-01) 47,233 GO Terms
Create an acyclic-directed graph using the GO.obo file.
Define remove_ancestors: Take a list of BP IDs and remove redundant ID ancestors. 
Define get_shared_bps_no_ancestors: Return the set of BP terms that two diseases share after removing redundant BP term ancestors.
Define count_elements: Count the number of GO IDs left (assumes that entries with zero elements are empty or null).
Define get_all_ancestors: Take a string of GO IDs and return a list containing the GO IDs and their parents.
Define count_elements: Count the number of GO IDs left (assumes that entries with zero elements are empty or null).


## Implement the BP-weight score function

In [19]:
def get_bp_weight_score(row1, row2):
    '''Return the BP-weight score between two diseases. The similarity
    score is calculated by summing the weight of the BP terms shared
    by both diseases. GO ID ancestors are removed from the set of 
    shared BP terms, so they do not affect the BP-weight. 
    
    Parameters:
    row1 (int): The row index number of disease 1.
    row2 (int): The row index number of disease 2.
    
    Notes: 
    The following variables are global variables. The function needs
    them in order to work. Passing variables as arguments is slower.
    
    bp_term_dict (dict): Stores row index keys and BP term list 
    values.
    weight (dict): Stores the weight of each BP term.
    '''
    # Create accumulator for summing weight of each shared BP term.
    total_weight = 0
    
    #  Get set of shared BP terms (the intersection).
    intersection = bp_term_dict[row1].intersection(bp_term_dict[row2])
        
    # Remove GO ID ancestors from list of shared BP terms.
    intersection = remove_ancestors(intersection)
        
    # Iterate thru every shared BP term.
    for bp in intersection:

        # Add weight of shared BP term.
        total_weight += weight[bp]
        
    # Return the weight of the shared BP terms.
    return total_weight

## Define a function that stores the BP-weight score for every disease combination

In [20]:
def store_bp_weight_scores(file):
    '''Store the BP-weight score between two diseases, for every 
    disease combination.
    
    Parameters:
    file: Pandas data frame storing a square matrix of every disease 
    combination.
    
    Notes: 
    The following variables are global variables. The function needs
    them in order to work. Passing variables as arguments is slower.
    
    bp_term_dict (dict): Stores row index keys and BP term list 
    values.
    weight (dict): Stores the weight of each BP term.
    '''
    # The number of rows is equal to number of keys in dictionary;
    # this is also equal to the number of diseases.
    row_count = len(bp_term_dict)
    
    # Iterate thru every row in the square matrix. 
    for row in range(0, row_count):
        
        # Iterate thru every column equal or larger than row index:
        # This will fill half of the matrix.
        for column in range(row, row_count):
                
            # Store shared BP-weight (the similarity score).
            file.at[row, column] = get_bp_weight_score(
                row, column)

## Store the BP-weight score without GO ID ancestors for every disease combination

This takes about 25 minutes.

In [21]:
# Fill square matrix with BP-weights (no GO ID ancestors).
store_bp_weight_scores(weight_score)

## Save the BP-weight score file as a .csv file

In [22]:
# Specify the filename
filename = 'BP-Weight.csv'

# Make index = True so that index columns aren't dropped.
weight_score.to_csv(filename, index = True)

# Note: Do not save square matrices as Excel sheets using 'to_excel' 
# function because saving matrices can take hours this way.
# Save matrices as .csv files and then manually convert them to Excel.